In [1]:
import torch
def print_device_status():
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print_device_status()

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
language_codes1 = [
    "eng_Latn",  # English written in Latin script
    "arb_Arab",  # Standard Arabic written in Arabic script
    "deu_Latn",  # German written in Latin script  
    "fra_Latn",  # French written in Latin script
    "guj_Gujr",  # Gujarati written in Gujarati script
    "hin_Deva",  # Hindi written in,evanagari script
    "jpn_Jpan",  # Japanese written in Japanese script
    "tam_Taml",  # Tamil written in Tamil script
    "pol_Latn",  # Polish written in Latin script
    "yue_Hant",  # Cantonese written in Traditional Chinese script - Cantonese
    "zho_Hans",  # Chinese written in Simplified Chinese script - Mandarin
    # "zho_Hant",  # Chinese written in Traditional Chinese script
    "spa_Latn",  # Spanish written in Latin script
    "tgl_Latn",  # Tagalog written in Latin script
    "urd_Arab",  # Urdu written in Arabic script
]



'''
Arabic D
German D
French D
Gujarati D
Hindi D
Japanese D
Tamil D
Polish D
Cantonese D
Mandarin D
Spanish D
Tagalog D
Urdu D
'''


'\nArabic D\nGerman D\nFrench D\nGujarati D\nHindi D\nJapanese D\nTamil D\nPolish D\nCantonese D\nMandarin D\nSpanish D\nTagalog D\nUrdu D\n'

### code to read result.csv (~17 lakh sentences) and filter out sentences with length less than 256 
### save this filtered csv into result_lt256.csv

In [3]:
# import pandas as pd
# from tqdm import tqdm

# df=pd.read_csv('result.csv') #results.csv
# df.head()
# print("original lenth :", len(df))

# filtered_rows = []

# for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
#     if len(str(row['English Sentences'])) < 256:
#         filtered_rows.append(row)

# filtered_df = pd.DataFrame(filtered_rows)

# print(filtered_df.head())
# print(filtered_df.describe())
# filtered_df.to_csv("result_lt256.csv", index=False)

In [3]:
import pandas as pd

filtered_df = pd.read_csv("result_lt256.csv")

print(len(filtered_df))
filtered_df.head()

1561567


,English Sentences
0,anyone else instead of sleeping more when depr...
1,idk how to elaborate on it i just started sudd...
2,to me it seems like an empty meaningless phras...
3,it s so pointless for me to still be alive my ...
4,i m starting to lose hope i feel like i m on a...


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load model and tokenizer
model_name = "facebook/nllb-200-distilled-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0): M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
          (fina

In [19]:
# from tqdm.auto import tqdm
# import gc
# import time
# import pandas as pd


# batch_size=16
# target_language_code = language_codes1

# def translate_sentences(sentences, tokenizer=tokenizer, model=model, device=device, batch_size=16, target_language_code = language_codes1):
#     df = pd.DataFrame(columns = target_language_code)

#     for i in tqdm(range(0, len(sentences), batch_size), desc="Translating"):
#         batch = sentences[i:i+batch_size]
#         # print(device)
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
#         for lang in target_language_code:
#             if lang == "eng_Latn":
#                 df[lang] = batch
            
#             else:
#                 with torch.no_grad():
#                     translated_ids = model.generate(
#                         inputs["input_ids"], 
#                         max_length=512, 
#                         forced_bos_token_id=tokenizer.lang_code_to_id[lang]
#                     )

#                 translated_batch = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)
#                 df[lang] = translated_batch

#                 gc.collect()
#                 torch.cuda.empty_cache()
            
#         df.to_csv(f"ouput_{i}_to_{i+batch_size}.csv")
    


In [5]:
sentences = filtered_df['English Sentences'].tolist()

sentences = sentences[:10]

In [7]:
from tqdm.auto import tqdm
import gc
import time
import pandas as pd


batch_size = 2
target_language_code = language_codes1


for i in tqdm(range(0, len(sentences), batch_size), desc="Translating"):
    df = pd.DataFrame(columns = target_language_code)


    batch = sentences[i:i+batch_size]
    # print(device)
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
    for lang in target_language_code:
        if lang == "eng_Latn":
            df[lang] = batch
        
        else:
            with torch.no_grad():
                translated_ids = model.generate(
                    inputs["input_ids"], 
                    max_length=512, 
                    forced_bos_token_id=tokenizer.lang_code_to_id[lang]
                )

            translated_batch = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)
            df[lang] = translated_batch

            gc.collect()
            torch.cuda.empty_cache()
    
    csv_file_name = f"ouput_{i}_to_{i+batch_size}.csv"
    df.to_csv(csv_file_name)
    print(f"{csv_file_name} created....................")
    


Translating:   0%|          | 0/5 [00:00<?, ?it/s]the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


In [ ]:
df = pd.DataFrame(columns = ["English", 
    "Arabic", 
    "German", 
    "French",
    "Gujarati",
    "Hindi",
    "Japanese",
    "Tamil",
    "Polish",
    "Cantonese",
    "Mandarin",
    "Spanish",
    "Tagalog",
    "Urdu",])